In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [4]:
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

train=train.drop(['id'],axis=1)
#test=test.drop(['id'],axis=1)

In [ ]:
train.columns[train.dtypes=='object']

In [ ]:
train[train['Weight Capacity (kg)']=='Nan']

In [ ]:
train['Brand'].unique()

In [5]:
train = train.replace(np.nan, "Na")


In [ ]:
train['Weight Capacity (kg)']=train['Weight Capacity (kg)'].replace("Nan",np.nan)

In [6]:
import catboost
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [7]:
X=train.drop(['Price'],axis=1)
y=train['Price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [8]:
# Initialize CatBoost Regressor
catboost_model = CatBoostRegressor(
    iterations=500,  # Number of boosting iterations
    learning_rate=0.1,  # Step size shrinkage
    depth=6,  # Maximum depth of trees
    loss_function='RMSE',  # Root Mean Squared Error for regression
    verbose=100,
    cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color']# Print training progress every 100 iterations
)

# Train the model
catboost_model.fit(X_train, y_train)


0:	learn: 39.0230899	total: 244ms	remaining: 2m 1s
100:	learn: 38.9680446	total: 10.6s	remaining: 41.7s
200:	learn: 38.9264625	total: 23.5s	remaining: 35s
300:	learn: 38.8931232	total: 36.4s	remaining: 24.1s
400:	learn: 38.8648935	total: 49.6s	remaining: 12.3s
499:	learn: 38.8377325	total: 1m 3s	remaining: 0us


In [9]:
preds=catboost_model.predict(X_test)

In [11]:
mean_squared_error(preds,y_test,squared=False)

39.03374079744069

In [12]:
test = test.replace(np.nan, "Na")

In [15]:
preds1=catboost_model.predict(test.drop(['id'],axis=1))

In [16]:
preds1

array([81.06875047, 82.53422753, 82.79295338, ..., 81.70742042,
       81.32063668, 80.81389181])

In [18]:
submission_df = pd.DataFrame({
    'id': test['id'],
    'Price':preds1
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

submission_df.head(5)

,id,Price
0,300000,81.068750
1,300001,82.534228
2,300002,82.792953
3,300003,81.526238
4,300004,78.476678
